# Assignment 6

### Part A

In [95]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [96]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [97]:
def exp_return(data, P):
    # Write your code here
    exp_ret = 0
    prices = data["Price in 1 Year"]
    prob = data["Probability"]
    
    for i in data.index:
        exp_ret += (prices[i] / P - 1) * prob[i]
    # exp_return(data, 40) should output 0.07500000000000002
    return exp_ret

In [98]:
exp_return(data, 40)

0.07500000000000002

Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [99]:
def volatility(data, P):
    # Write your code here
    exp_ret = exp_return(data, P)
    var = 0
    vol = 0
    prices = data["Price in 1 Year"]
    prob = data["Probability"]
    for i in data.index:
        var += prob[i] * ((prices[i] / P - 1) - exp_ret) ** 2

    vol = var ** (1/2)

    # volatility(data,40) should output 0.21301408404140795
    return vol

In [100]:
volatility(data, 40)

0.21301408404140795

Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [101]:
def interval(data, P):
    # Write your code here
    prices = data["Price in 1 Year"]
    prob = data["Probability"]
    
    R = exp_return(data, P)
    SD = volatility(data, P)

    SE = SD / (3**(1/2))
    upper = R + 1.96 * SE
    lower = R - 1.96 * SE
    ret = [lower, upper]
    # ret[0] = 
    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]
    return ret

In [102]:
interval(data, 40)

[-0.16604812797447732, 0.3160481279744774]

Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [103]:
def exp_return_capm(beta, market_return, rf):
    # Write your code here

    exp_capm = rf + beta * (market_return - rf)

    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028
    return exp_capm

In [104]:
exp_return_capm(-0.6,0.1, 0.02)

-0.028

### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [105]:
def clean_data(path):
    # Write your code here
    # This should yield data with 403316 rows × 4 columns

    #original = 4344733 rows

    df = pd.read_csv(path)
    df["RET"] = pd.to_numeric(df["RET"], errors='coerce')
    df = df.dropna()

    item_counts = df["PERMNO"].value_counts()

    filtered_df = df[df["PERMNO"].isin(item_counts.index[item_counts >= 500])]

    return filtered_df


In [106]:
clean_data("monthlycrsp_mod6.csv")

PERMNO        date     CUSIP       RET
9027      10065  1970-01-30  00621210 -0.144375
9028      10065  1970-02-27  00621210  0.105882
9029      10065  1970-03-31  00621210  0.017857
9030      10065  1970-04-30  00621210 -0.078947
9031      10065  1970-05-29  00621210 -0.020952
...         ...         ...       ...       ...
3393558   84129  2022-08-31  98970110  0.016315
3393559   84129  2022-09-30  98970110 -0.075777
3393560   84129  2022-10-31  98970110  0.021235
3393561   84129  2022-11-30  98970110  0.005583
3393562   84129  2022-12-30  98970110 -0.051332

[403316 rows x 4 columns]

Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [107]:
def avg_return(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns

    unique_sec = data["PERMNO"].unique()

    grouped = data.groupby("PERMNO")
    avg_returns = grouped["RET"].mean()
    
    output = pd.DataFrame(data = {
        "CUSIP" : unique_sec,
        "Average Return" : avg_returns
    })
    
    return output

In [108]:
avg_return(clean_data("monthlycrsp_mod6.csv"))

CUSIP  Average Return
PERMNO                       
10065   10065        0.009364
10145   10145        0.012476
10225   10225        0.013279
10516   10516        0.014130
10656   10656        0.009859
...       ...             ...
81614   81614        0.015500
81702   81702        0.015024
82107   82107        0.011280
83601   83601        0.016190
84129   84129        0.012356

[679 rows x 2 columns]

In [109]:
def security_volatilities(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns

    unique_sec = data["PERMNO"].unique()

    grouped = data.groupby("PERMNO")
    var = grouped["RET"].var()
    vol = var**(1/2)
    
    output = pd.DataFrame(data = {
        "CUSIP" : unique_sec,
        "Volatility" : vol
    })
    
    return output

In [110]:
security_volatilities(clean_data("monthlycrsp_mod6.csv"))

CUSIP  Volatility
PERMNO                   
10065   10065    0.049088
10145   10145    0.080908
10225   10225    0.072392
10516   10516    0.084347
10656   10656    0.117598
...       ...         ...
81614   81614    0.167238
81702   81702    0.111982
82107   82107    0.073001
83601   83601    0.103132
84129   84129    0.095243

[679 rows x 2 columns]

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [111]:
def correlation(ret_data, vol_data):
    # Write your code here
    # correlation(rets, vols) should yield 0.44837090727059614

    corr = np.corrcoef(ret_data["Average Return"], vol_data["Volatility"])[0, 1]
    
    return corr

In [112]:
correlation(avg_return(clean_data("monthlycrsp_mod6.csv")), security_volatilities(clean_data("monthlycrsp_mod6.csv")))

0.44837090727059614